# Sprint 深層学習スクラッチ　リカレンとニューラルネットワーク

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#このSprintについて" data-toc-modified-id="このSprintについて-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>このSprintについて</a></span><ul class="toc-item"><li><span><a href="#Sprintの目的" data-toc-modified-id="Sprintの目的-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Sprintの目的</a></span></li><li><span><a href="#どのように学ぶか" data-toc-modified-id="どのように学ぶか-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>どのように学ぶか</a></span></li></ul></li><li><span><a href="#リカレントニューラルネットワークスクラッチ" data-toc-modified-id="リカレントニューラルネットワークスクラッチ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>リカレントニューラルネットワークスクラッチ</a></span><ul class="toc-item"><li><span><a href="#【問題1】SimpleRNNのフォワードプロパゲーション実装" data-toc-modified-id="【問題1】SimpleRNNのフォワードプロパゲーション実装-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>【問題1】SimpleRNNのフォワードプロパゲーション実装</a></span></li><li><span><a href="#【問題2】小さな配列でのフォワードプロパゲーションの実験" data-toc-modified-id="【問題2】小さな配列でのフォワードプロパゲーションの実験-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>【問題2】小さな配列でのフォワードプロパゲーションの実験</a></span></li></ul></li></ul></div>

## このSprintについて
### Sprintの目的
* スクラッチを通してリカレントニューラルネットワークの基礎を理解する

### どのように学ぶか
スクラッチでリカレントニューラルネットワークの実装を行います。

## リカレントニューラルネットワークスクラッチ
リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

### 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

In [28]:
import numpy as np
import matplotlib.pyplot as plt

In [53]:
class Tanh():
    """
    活性化関数 : ハイパボリックタンジェント関数
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        self.A = A
        self.Z = np.tanh(self.A)
        return self.Z
    
    def backward(self,dZ):
        return dZ*(1-self.Z**2)

In [54]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
        
    def Wx(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W : 重み
        """
        #self.sigma * np.random.randn(n_nodes1, n_nodes2)
        
        Wx = np.array([[1, 3, 5, 7],
                       [3, 5, 7, 8]])/100 # 問題2のミニテスト用Wx
        return Wx
    
    def Wh(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W : 重み
        """
        #self.sigma * np.random.randn(n_nodes1, n_nodes2)
        
        Wh = np.array([[1, 3, 5, 7],
                       [2, 4, 6, 8],
                       [3, 5, 7, 8],
                       [4, 6, 8, 10]])/100 # 問題2のミニテスト用Wh
        return Wh
    
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B : バイアス
        """
        #np.zeros(n_nodes2)
        B = np.array([1, 1, 1, 1]) # 問題2のミニテスト用バイアス
        return B

In [63]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_batch, input_nodes, output_nodes, initializer, activation):
        
        self.n_batch = n_batch
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.initializer = initializer
        self.activation = activation
        
        # 初期化
        # initializerのメソッドで、self.Wx,self.Wh,self.Bを初期化する
        
        self.Wx = self.initializer.Wx(self.input_nodes, self.output_nodes)
        self.Wh = self.initializer.Wh(self.output_nodes, self.output_nodes)
        self.B = self.initializer.B(self.output_nodes)
        
        self.H = np.zeros([self.n_batch, self.output_nodes])
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.X = X
        self.A = np.dot(self.X,self.Wx) +np.dot(self.H,self.Wh) +self.B
        self.H = self.activation.forward(self.A)
        
        return self.H

In [56]:
class ScratchSimpleRNNClassifier:
    
    def __init__(self, n_nodes):
        """
        Parameters
        ----------
        """
        self.n_nodes = n_nodes
        
    
    def fit(self, X):
        """
        Parameters
        ----------
        """
        self.n_batch = X.shape[0]
        self.n_features = X.shape[2]
        
        initializer = SimpleInitializer()
        
        affine = FC(self.n_batch, self.n_features, self.n_nodes, initializer, Tanh())
        
        # forward
        for t in range(X.shape[1]):
            h = affine.forward(X[:,t,:])
            
        return h

In [57]:
# (batch_size, n_sequences, n_features)
x = np.array([[[1, 2],
               [2, 3],
               [3, 4]]])/100 

 # (n_features, n_nodes)
w_x = np.array([[1, 3, 5, 7],
                [3, 5, 7, 8]])/100

# (n_nodes, n_nodes)
w_h = np.array([[1, 3, 5, 7],
                [2, 4, 6, 8],
                [3, 5, 7, 8],
                [4, 6, 8, 10]])/100 

batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4

h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

### 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。

入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [64]:
rnn = ScratchSimpleRNNClassifier(4)
print(rnn.fit(x))

[[0.79494228 0.81839002 0.83939649 0.85584174]]
